In [1]:
import os

root = "model_assets/jvnv-F1"
pth_files = [os.path.join(root, f) for f in os.listdir(root) if f.endswith(".pth")]

In [8]:
import torch

model_name = "G_0.pth"
model = torch.load(f"pretrained/{model_name}", map_location=torch.device("cpu"))

In [5]:
from safetensors.torch import save_file
state_dict = model["model"]
new_dict = {}
for k in state_dict.keys():
    if k.startswith("enc_p.emo"):
        print(k)
    else:
        new_dict[k] = state_dict[k]


enc_p.emo_proj.bias
enc_p.emo_q_proj.weight
enc_p.emo_q_proj.bias


In [6]:
save_file(new_dict, f"pretrained/{model_name.replace('.pth', '.safetensors')}")

In [1]:
from glob import glob
root_dir = "model_assets"

safetensors_files = glob(f"{root_dir}/**/*.safetensors", recursive=True)


In [ ]:
# 推論に不要なenc_qを消し忘れていたのを削除

from safetensors import safe_open
from safetensors.torch import save_file

for path in safetensors_files:
    print(path)
    tensors = {}
    with safe_open(path, framework="pt", device="cpu") as f:
        for key in f.keys():
            if key.startswith("enc_q"):
                print(key)
                continue
            tensors[key] = f.get_tensor(key)
    save_file(tensors, path.replace(".safetensors", ".new.safetensors"))

In [6]:
# pthファイルを推論用safetensorsに変換
from safetensors.torch import save_file
from safetensors import safe_open
import torch

pth_path = "model_assets/jvnv-F1/release_7000.pth"
pth_weight = torch.load(pth_path, map_location=torch.device("cpu"))
new_dict = {}
for key in pth_weight["model"]:
    if key.startswith("enc_p.xvec_proj."):  # 前のモデルの名残
        print(key, key.replace("enc_p.xvec_proj.", "enc_p.style_proj."))
        new_dict[key.replace("enc_p.xvec_proj.", "enc_p.style_proj.")] = pth_weight["model"][key].clone().contiguous()  # よく分からないおまじないをしないとエラーになる
    elif not key.startswith("enc_q"):
        new_dict[key] = pth_weight["model"][key]
    else:
        continue
new_dict["iteration"] = torch.LongTensor([pth_weight["iteration"]])
save_file(new_dict, pth_path.replace(".pth", ".pth.safetensors"))


enc_p.xvec_proj.weight enc_p.style_proj.weight
enc_p.xvec_proj.bias enc_p.style_proj.bias
